In [ ]:
# In[0]: Load Packages
import torch

In [ ]:
# In[1]: Basic tensor operations

In [ ]:
## 1D Tensors

In [ ]:
# Construct LongTensor
a = torch.tensor([1, 2, 3, 4])
print(a.type())  # type of Tensor
print(a.dtype)  # type of data within Tensor

In [ ]:
# Methods to construct float Tensor
a = torch.tensor([1, 2, 3], dtype=float)  # float64
a = torch.FloatTensor([1, 2, 3])  # float32
a = a.type(torch.FloatTensor)  # transform tensor into float32

In [ ]:
# Informative methods
a.size()  # == a.shape
a.ndimension()  # rank

In [ ]:
# Transformative methods
a_col = a.view()